In [1]:
import pandas as pd
import json



In [23]:
# Load the predictions from the CSV file
preds_df = pd.read_csv('/home/ashrafs/projects/dragon/runs/fever/dragon__fever_ih_false_load__elr1e-5_dlr1e-3_W100_b128_ufz2_e22_sd5__20240226_112825/test_e9_preds.csv', header=None)

# Load the test data from the JSONL file and convert it to a DataFrame
test_data = []
with open('/home/ashrafs/projects/dragon/data/fever/Main/test.jsonl', 'r') as file:
    for line in file:
        test_data.append(json.loads(line))
test_df = pd.DataFrame(test_data)

# Display the first few rows of both DataFrames to understand their structure
preds_df.head(), test_df.head()


(       0  1
 0  12969  B
 1   2088  A
 2  12899  A
 3  18540  B
 4  31292  A,
       id                                           question answerKey
 0  12969  {'stem': 'Lianyun District in the Jiangsu Prov...         B
 1   2088  {'stem': 'Sudhin Das (Bangladeshi musician) wa...         B
 2  12899  {'stem': 'Pikahsso has released twelve singles...         B
 3  18540  {'stem': 'Georges Cadoudal, remaining true to ...         A
 4  31292  {'stem': 'Cân i Gymru 2010 had eight contestan...         B)

In [24]:
test_df['answerKey'].value_counts()

answerKey
A    1944
B    1753
C     248
Name: count, dtype: int64

In [25]:
# Rename columns in preds_df for clarity
preds_df.columns = ['id', 'predictedLabel']

# Ensure 'id' columns are of the same type to avoid merge issues
preds_df['id'] = preds_df['id'].astype(int)
test_df['id'] = test_df['id'].astype(int)

# Merge the predictions with the actual labels on 'id'
merged_df = pd.merge(test_df, preds_df, on='id')

# Display the first few rows of the merged DataFrame to confirm the merge
merged_df.head()


,id,question,answerKey,predictedLabel
0,12969,{'stem': 'Lianyun District in the Jiangsu Prov...,B,B
1,2088,{'stem': 'Sudhin Das (Bangladeshi musician) wa...,B,A
2,12899,{'stem': 'Pikahsso has released twelve singles...,B,A
3,18540,"{'stem': 'Georges Cadoudal, remaining true to ...",A,B
4,31292,{'stem': 'Cân i Gymru 2010 had eight contestan...,B,A


,12969
count,3944.000000
mean,14588.012677
std,13269.338697
min,5.000000
25%,3886.500000
50%,9816.000000
75%,22405.250000
max,52448.000000


In [26]:
len(preds_df), len(test_df)

(3945, 3945)

In [27]:
from sklearn.metrics import classification_report

# Extract actual and predicted labels
y_true = merged_df['answerKey']
y_pred = merged_df['predictedLabel']

# Compute the classification report
class_report = classification_report(y_true, y_pred, target_names=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"])

print(class_report)


                 precision    recall  f1-score   support

       SUPPORTS       0.59      0.82      0.69      1944
        REFUTES       0.65      0.47      0.55      1753
NOT ENOUGH INFO       0.00      0.00      0.00       248

       accuracy                           0.61      3945
      macro avg       0.41      0.43      0.41      3945
   weighted avg       0.58      0.61      0.58      3945



/home/ashrafs/miniconda3/envs/dragon2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ashrafs/miniconda3/envs/dragon2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ashrafs/miniconda3/envs/dragon2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [28]:
from sklearn.metrics import f1_score

# Calculate the average F1 score
average_f1 = f1_score(y_true, y_pred, average='weighted')

average_f1


0.5815216455582145